In [1]:
from keras.utils import np_utils
import matplotlib.pyplot as plt 
from sklearn.datasets import load_files       
import numpy as np
from glob import glob
from keras.layers import *
from keras.models import *
from keras.applications import *
from keras import layers
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from PIL import Image

/home/dupad/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
import sys
print(sys.version)

3.6.5 |Anaconda, Inc.| (default, Apr 29 2018, 16:14:56) 
[GCC 7.2.0]


In [3]:
import tensorflow as tf
import keras

In [4]:
print(tf.__version__)

1.8.0


In [5]:
import os
os.chdir('/home/dupad')

In [6]:
batch_size = 8

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        rotation_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        cval=0.2,
        horizontal_flip=True,
        vertical_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'autoimages/t1',  # this is the target directory
        target_size=(299,299),  # all images will be resized to 299*299
        batch_size=batch_size,
        class_mode='categorical')  # since we use binary_crossentropy loss, we need binary labels
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'autoimages/v1',
        target_size=(299,299),
        batch_size=batch_size,
        class_mode='categorical')

Found 3910 images belonging to 5 classes.
Found 922 images belonging to 5 classes.


In [11]:
def  get_inception_v3(nb_classes,drop):
    inception = InceptionV3(weights="imagenet", pooling='avg', include_top=False)

    x = inception.output
    if drop:
        x = Dropout(0.8)(x)
        x = Dense(1024, activation='relu')(x)  # Why 1024 are not more or less is not clear... Should be tuned if possible
        x = Dropout(0.5)(x)
    else:
        x = Dense(1024, activation='relu')(x)
    predictions = Dense(nb_classes, activation='softmax')(x)

    # This is the model we will train
    model = Model(inputs=inception.input, outputs=predictions)

    return model

In [13]:
nb_classes = 5
drop=False
model = get_inception_v3(nb_classes,drop)

In [10]:
for layer in model.layers[:-20]:
   layer.trainable = False
for layer in model.layers[-20:]:
   layer.trainable = True

In [11]:
from keras import regularizers

In [12]:
model.layers[-20].get_config()

{'name': 'conv2d_86',
 'trainable': True,
 'filters': 320,
 'kernel_size': (1, 1),
 'strides': (1, 1),
 'padding': 'same',
 'data_format': 'channels_last',
 'dilation_rate': (1, 1),
 'activation': 'linear',
 'use_bias': False,
 'kernel_initializer': {'class_name': 'VarianceScaling',
  'config': {'scale': 1.0,
   'mode': 'fan_avg',
   'distribution': 'uniform',
   'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [13]:
model.layers[-1].kernel_regularizer=regularizers.l2(0.01)

In [14]:
model.layers[-2].kernel_regularizer=regularizers.l2(0.01)

In [15]:
model.layers[-15].kernel_regularizer=regularizers.l2(0.01)

In [16]:
model.layers[-20].kernel_regularizer=regularizers.l2(0.1)

In [17]:
print(model.layers[-1].kernel_regularizer)

In [14]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d_95 (Conv2D)              (None, None, None, 3 864         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_95 (BatchNo (None, None, None, 3 96          conv2d_95[0][0]                  
__________________________________________________________________________________________________
activation_95 (Activation)      (None, None, None, 3 0           batch_normalization_95[0][0]     
__________________________________________________________________________________________________
conv2d_96 

In [15]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

from keras import backend as K
K.tensorflow_backend._get_available_gpus()

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12235611101937562223
]


[]

In [16]:
from keras.callbacks import ModelCheckpoint  

### TODO: specify the number of epochs that you would like to use to train the model.

epochs = 5

### Do NOT modify the code below this line.

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.from_scratch.hdf5', 
                               verbose=1, save_best_only=True)

model.fit_generator(train_generator,
        steps_per_epoch=500// batch_size,
        epochs=epochs,shuffle=True, 
        validation_data=validation_generator,
        validation_steps=60// batch_size,
        verbose=1)

Epoch 1/5
62/62 [==============================] - 659s 11s/step - loss: 0.8159 - acc: 0.7802 - val_loss: 0.7859 - val_acc: 0.8036
Epoch 2/5
62/62 [==============================] - 654s 11s/step - loss: 0.9069 - acc: 0.7440 - val_loss: 0.8092 - val_acc: 0.8036
Epoch 3/5
62/62 [==============================] - 634s 10s/step - loss: 0.6811 - acc: 0.8259 - val_loss: 0.7713 - val_acc: 0.8036
Epoch 4/5
62/62 [==============================] - 637s 10s/step - loss: 0.7362 - acc: 0.8044 - val_loss: 0.7589 - val_acc: 0.8036
Epoch 5/5
62/62 [==============================] - 11125s 179s/step - loss: 0.7531 - acc: 0.8004 - val_loss: 0.7434 - val_acc: 0.8036


In [30]:
# model.save('saved_models/weights.best.from_scratch_June18.hdf5')

In [55]:
model.load_weights('saved_models/weights.best.from_scratch_June18.hdf5')

OSError: Unable to open file (unable to open file: name = 'saved_models/weights.best.from_scratch_June18.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [57]:
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import scipy
from matplotlib.pyplot import imshow